# Tipos de agentes

Para definir los diferentes tipos de agentes, necesitamos definir un ambiente. En este caso usaremos el problema de la aspiradora, pero crearemos un ambiente en donde se cuente con cuatro cuadrados:

$$
\begin{pmatrix}
A & B \\ C & D
\end{pmatrix}
$$

Los movimientos que se podrán realizar son, entonces, a los lados (izquierda, derecha) y hacia arriba o abajo.

In [22]:
import random

class Square(object):
    def __init__(self, name):
        """
        Crea un objeto square, que representa los cuadrados del ambiente.

        Argumentos
        ----------
        name : str
          Nombre del cuadrado ('A', 'B', etc.)
        """
        self.name = name
        
        # Hasta que no se coloque con cuadrados vecinos, los movimientos no llevan a ningún lado
        self.left = self
        self.right = self
        self.up = self
        self.down = self
        
class VacuumWorld(object):
    def __init__(self, dirt_init='random'):
        """
        Objeto que crea el ambiente para el mundo de la aspiradora:
        Se conforma de:
        - Cuatro cuadrados: A, B, C y D
        - Indicación de la limpieza (0) o suciedad (1) de los cuadrados.

        Argumentos
        ----------
        dirt_init : str
          Forma en que se inicializará la suciedad de los cuadrados.
        init_loc : str
          Cuadrado en que se iniciará el agente (A o B).
        """
        self.squares = []
        self.A = Square('A')
        self.B = Square('B')
        self.C = Square('C')
        self.D = Square('D')
        self.squares.append(self.A)
        self.squares.append(self.B)
        self.squares.append(self.C)
        self.squares.append(self.D)

        #Cambia la posición de A
        self.A.right = self.B
        self.A.down = self.C
        #Cambia la posición de B
        self.B.left = self.A
        self.B.down = self.D
        #Cambia la posición de C
        self.C.right = self.D
        self.C.up = self.A
        #Cambia la posición de D
        self.D.left = self.C
        self.D.up = self.B

        #Inicialización de suciedad
        self.dirt_init = dirt_init
        
    def initialize_dirt(self):
        """
        Inicializa la suciedad de los cuadrados:

        Opciones (se indican en init)
        --------
        random : Inicializa con uno de los cuadrados sucios y otro limpio de forma aleatoria.
        dirty : Inicializa con los dos cuadrados sucios
        clean : Inicializa con los dos cuadrados limpios
        """
        if self.dirt_init=='random':
            for square in self.squares:
                if random.random() > 0.5:
                    square.dirt = 1
                else:
                    square.dirt = 0

        elif self.dirt_init=='dirty':
            for square in self.squares:
                square.dirt = 1

        elif self.dirt_init=='clean':
            for square in self.squares:
                square.dirt = 0

        else:
            for square, value in zip(self.squares, self.dirt_init):
                square.dirt = value
                    
    def initialize_agent_location(self, agent):
        """
        Inicializa la localización de un agente dado el objeto agente.

        Argumentos
        ----------
        agent : object
          Objeto agente que interecturá con el ambiente.
        """
        #Inicializa al objeto de forma aleatoria
        i = random.randint(0, len(self.squares)-1)
        agent.location = self.squares[i]

Iniciamos el ambiente de tal forma que la suciedad se distributa de manera aleatoria.

In [332]:
#Creación del ambiente
env = VacuumWorld(dirt_init='random')

#Inicializa la suciedad
env.initialize_dirt()

print('Inicialización de estados:\n{}'.format([(s.name, s.dirt) for s in env.squares]))

Inicialización de estados:
[('A', 0), ('B', 1), ('C', 1), ('D', 0)]


## Agente dirigido mediante tabla

El primer agente que definiremos es un agente dirigido mediante tabla. En este caso, definimos al agente, el cual requiere de una tabla que especifique las acciones que debe tomar según la secuencia de percepciones.

In [301]:
class TableDrivenAgent(VacuumWorld):
    """
    Objeto agente.
    """
    def __init__(self, table):
        """
        percept
            Percepción del agente, enlista las percepciones hasta el estado actual
        table
            Tabla de percepción-acción
        location
            Lugar donde se localiza el agente
        """
        self.percept = None
        self.table = table
        self.location = None
        
    def sensor(self):
        """
        Función que determina la percepción a partir de observar 
        en donde se encuentra el agente.
        """
        if self.percept == None:
            self.percept = (self.location.name, self.location.dirt)
        else:
            self.percept += (self.location.name, self.location.dirt)
        
    def decide(self):
        """
        Simple función para ejecutar una acción a partir de 
        consultar la tabla
        """
        action = self.table[self.percept]
        #Cuando se tiene más de una acción
        #la acción se realiza aleatoriamente
        if type(action) == list:
            action = random.choice(action)
        
        return action

Definimos la tabla, que en este caso sólo cuenta con sólo combinaciones de hasta dos elementos. Una tabla más completa consideraría un mayor número de combinaciones (de hasta cuatro elementos, para que el agente pueda explorar todo el ambiente).

In [330]:
table = {('A',1):'Clean', ('A',0): ['Right', 'Down'], ('B',1):'Clean', ('B',0): ['Left', 'Down'],
        ('C',1):'Clean', ('C',0): ['Right', 'Up'], ('D',1):'Clean', ('D',0): ['Left', 'Up'],
        (('A',0,'B',0)):'Down', (('A',0,'C',0)):'Right',
        (('A',0,'B',1)):'Clean', (('A',0,'C',1)):'Clean',
        (('A',1,'B',0)):'Down', (('A',1,'C',0)):'Right', (('A',1,'A',0)):['Right', 'Down'],
        (('B',0,'A',0)):'Down', (('B',0,'D',0)):'Left',
        (('B',0,'A',1)):'Clean', (('B',0,'D',1)):'Clean',
        (('B',1,'A',0)):'Down', (('B',1,'D',0)):'Left', (('B',1,'B',0)):['Left', 'Down'],
        (('C',0,'A',0)):'Right', (('C',0,'D',0)):'Up',
        (('C',0,'A',1)):'Clean', (('C',0,'D',1)):'Clean',
        (('C',1,'A',0)):'Right', (('C',1,'D',0)):'Up', (('C',1,'C',0)):['Right', 'Up'],
        (('D',0,'B',0)):'Left', (('D',0,'C',0)):'Up',
        (('D',0,'B',1)):'Clean', (('D',0,'C',1)):'Clean',
        (('D',1,'B',0)):'Left', (('D',1,'C',0)):'Up', (('D',1,'D',0)):['Left', 'Up']}

Creamos el agente, y le asignamos la tabla que hemos definido. Inicializamos su posición en el ambiente.

In [331]:
#Creamos el agente
agent = TableDrivenAgent(table)
#Inicializamos la localización del agente
env.initialize_agent_location(agent)

print('Posición actual del agente: {}'.format(agent.location.name))

Posición actual del agente: A


In [297]:
#agent.table

In [329]:
for t in range(0,2):
    #El agente revisa el ambiente
    agent.sensor()
    #En base a su percepción decide por una acción
    action = agent.decide()
    
    #Con base en la acción, se hace un cambio
    if action == 'Clean':
        agent.location.dirt = 0
    if action == 'Left':
        agent.location = agent.location.left
    if action == 'Right':
        agent.location = agent.location.right
    if action == 'Up':
        agent.location.up
    if action == 'Down':
        agent.location = agent.location.down
        
    #Se reinician las percpeciones
    #Si se tiene una tabla completa, no es necesario
    agent.percept = None

        
    print('Tiempo: {}\nAcción: {}\nNueva localización: {}\n'.format(t,action,agent.location.name))
    
print('Estados al final del proceso:\n{}'.format([(s.name, s.dirt) for s in env.squares]))

Tiempo: 0
Acción: Left
Nueva localización: C

Tiempo: 1
Acción: Right
Nueva localización: D

Estados al final del proceso:
[('A', 0), ('B', 0), ('C', 0), ('D', 0)]


## Agente reactivo simple

In [333]:
class SimpleReflexAgent(object):
    """
    Objeto agente.
    """
    def __init__(self):
        self.location = None
        
    def decide(self):        
        if self.location.dirt == 1:
            self.location.dirt = 0
        elif self.location.name == 'A':
            self.location = random.choice([self.location.right, self.location.down])
        elif self.location.name == 'B':
            self.location = random.choice([self.location.left, self.location.down])
        elif self.location.name == 'C':
            self.location = random.choice([self.location.right, self.location.up])
        elif self.location.name == 'D':
            self.location = random.choice([self.location.left, self.location.up])

In [334]:
#Creamos el agente
sr_agent = SimpleReflexAgent()
#Inicializamos la localización del agente
env.initialize_agent_location(sr_agent)

print('Posición actual del agente: {}'.format(sr_agent.location.name))

Posición actual del agente: A


In [335]:
action = sr_agent.decide()
print(action)
print('Posición actual del agente: {}'.format(sr_agent.location.name))

None
Posición actual del agente: C
